In [1]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144
import pandas as pd
import numpy as np

# ML: Predicting Star Ratings


Our objective is to predict a new venue's popularity from information available when the venue opens.  We will do this by machine learning from a data set of venue popularities provided by Yelp.  The data set contains meta data about the venue (where it is located, the type of food served, etc.).  It also contains a star rating. Note that the venues are not limited to restaurants.


## Metric


Our model will be assessed based on the root mean squared error of the number of stars predicted. 


## Download and parse the incoming data


We start by downloading the data set from Amazon S3:

In [ ]:
!aws s3 sync s3://dataincubator-course/mldata/ . --exclude '*' --include 'yelp_train_academic_dataset_business.json.gz'

The training data are a series of JSON objects, in a Gzipped file. Python supports Gzipped files natively: [`gzip.open`](https://docs.python.org/2/library/gzip.html) has the same interface as `open`, but handles `.gz` files automatically.

The built-in json package has a `loads()` function that converts a JSON string into a Python dictionary.  We could call that once for each row of the file. [`ujson`](http://docs.micropython.org/en/latest/library/ujson.html) has the same interface as the built-in `json` library, but is *substantially* faster (at the cost of non-robust handling of malformed json).  We will use that inside a list comprehension to get a list of dictionaries:

In [4]:
import ujson as json
import gzip

with gzip.open('yelp_train_academic_dataset_business.json.gz') as f:
    data = [json.loads(line) for line in f]

In Scikit Learn, the labels to be predicted, in this case, the stars, are always kept in a separate data structure than the features.

In [5]:
datapd= pd.DataFrame.from_dict(data)

In [6]:
#number of rows of the dataset
len(datapd)

37938

In [7]:
#Separating the labels
star_ratings = [row['stars'] for row in data]

### Notes:

There are obvious mistakes in the data.  There is no need to try to correct them at this point

## Building models


we will build and train different estimator that predicts the star rating given certain features. In most cases we will use a pipeline containing custom or pre-built transformers and an existing estimator.


Also in some models we find it useful to serialize the trained models to disk.  This will allow to reload it after restarting the Jupyter notebook, without needing to retrain it.  We will be using the [`dill` library](https://pypi.python.org/pypi/dill) for this (although the [`joblib` library](http://scikit-learn.org/stable/modules/model_persistence.html) also works). We will Use
```python
dill.dump(estimator, open('estimator.dill', 'w'))
estimator = dill.load(open('estimator.dill', 'r'))
```

# Models


## city_avg

The venues belong to different cities.  You can imagine that the ratings in some cities are probably higher than others.  We wish to build an estimator to make a prediction based on this, but first we need to work out the average rating for each city.  For this problem, we create a list of tuples (city name, star rating), one for each city in the data set.


In [12]:
from collections import defaultdict
star_sum = defaultdict(int)
count = defaultdict(int)
avg_star = {}

for row, stars in zip(data, star_ratings):
    count[row["city"]] +=1
    star_sum[row["city"]] += stars

for city in star_sum:
    avg_star[city] = star_sum[city]/count[city]

In [13]:
avg_star

{'Phoenix': 3.6702903946388683,
 'De Forest': 3.75,
 'Mc Farland': 3.1,
 'Middleton': 3.611111111111111,
 'Madison': 3.6457337883959045,
 'Sun Prairie': 3.455223880597015,
 'Windsor': 3.5,
 'Monona': 3.4727272727272727,
 'Chandler': 3.667574931880109,
 'Scottsdale': 3.8206757594544327,
 'Tempe': 3.644621295279912,
 'Florence': 3.6176470588235294,
 'Peoria': 3.6388367729831144,
 'Glendale': 3.607404021937843,
 'Cave Creek': 3.9122137404580153,
 'Paradise Valley': 3.6690140845070425,
 'Mesa': 3.5901461829994585,
 'Ahwatukee': 3.6875,
 'Pheonix': 3.0,
 'Anthem': 3.7818181818181817,
 'Gilbert': 3.752396166134185,
 'Gold Canyon': 3.5,
 'Apache Junction': 3.6375,
 'Goldfield': 3.5,
 'Casa Grande': 3.5172413793103448,
 'Coolidge': 3.4375,
 'Higley': 3.5,
 'Queen Creek': 3.6456043956043955,
 'Sun Lakes': 3.2222222222222223,
 'Goodyear': 3.5313653136531364,
 'Fort Mcdowell': 4.0,
 'Fountain Hills': 3.7904761904761903,
 'Fountain Hls': 3.0,
 'Maricopa': 3.52,
 'chandler': 5.0,
 'Buckeye': 3.4084

Now we can calculate the average ratings.  Again, a dictionary makes a good container.

In [14]:
avgdata = pd.DataFrame(datapd.groupby(["city"]).mean())
avg_stars = avgdata["stars"].to_dict()
avgdata = avgdata.reset_index()
datapd = datapd.merge(avgdata[["city","stars"]],  on="city")
datapd["avgstars_city"] = datapd["stars_y"]
datapd = datapd.drop(["stars_y"], axis=1)

In [ ]:
datapd["avgstars_city"] = datapd["stars_y"]

In [15]:
datapd.head()

,attributes,business_id,categories,city,full_address,hours,latitude,longitude,name,neighborhoods,open,review_count,stars_x,state,type,avgstars_city
0,{'By Appointment Only': True},vcNAWiLM4dR7D2nwwJ7nCA,"[Doctors, Health & Medical]",Phoenix,"4840 E Indian School Rd\nSte 101\nPhoenix, AZ ...","{'Tuesday': {'close': '17:00', 'open': '08:00'...",33.499313,-111.983758,"Eric Goldberg, MD",[],True,7,3.5,AZ,business,3.67029
1,"{'Take-out': True, 'Wi-Fi': 'no', 'Alcohol': '...",x5Mv61CnZLohZWxfCVCPTQ,"[Sandwiches, Pizza, Chicken Wings, Restaurants]",Phoenix,"2819 N Central Ave\nPhoenix, AZ 85004",{},33.479542,-112.073418,Domino's Pizza,[],True,12,2.5,AZ,business,3.67029
2,"{'Take-out': True, 'Noise Level': 'quiet', 'De...",2ZnCITVa0abGce4gZ6RhIw,"[American (New), Sandwiches, Restaurants]",Phoenix,"1850 N Central Ave\nPhoenix, AZ 85004",{},33.468988,-112.074315,Viad Tower Restaurants,[],True,5,3.5,AZ,business,3.67029
3,"{'Alcohol': 'full_bar', 'Price Range': 1, 'Noi...",EmzaQR5hQlF0WIl24NxAZA,"[American (New), Nightlife, Dance Clubs, Resta...",Phoenix,"132 E Washington St\nPhoenix, AZ 85004","{'Sunday': {'close': '02:00', 'open': '21:00'}...",33.448399,-112.071702,Sky Lounge,[],True,20,2.5,AZ,business,3.67029
4,"{'Price Range': 2, 'Alcohol': 'full_bar', 'Goo...",SiwN7f0N4bs4ZtPc4yPgiA,"[Nightlife, Dance Clubs]",Phoenix,"710 N Central Ave\nPhoenix, AZ 85004",{},33.456068,-112.074225,Palazzo,[],True,15,2.5,AZ,business,3.67029


In [16]:
sum(star_ratings)/len(star_ratings)

3.6729137013021247

##### Check point
There should be 167 different cities:

In [17]:
assert len(avg_star) == 167

## city_model

Now, let's build a custom estimator that will make a prediction based solely on the city of a venue.  

In [18]:
from sklearn import base
from collections import defaultdict

class CityEstimator(base.BaseEstimator, base.RegressorMixin):
    
    def __init__(self):
        self.avg_stars = {}
    
    def fit(self, X, y):
        star_sum = defaultdict(int)
        count = defaultdict(int)
        for row, stars in zip(X, y):
            count[row["city"]] +=1
            star_sum[row["city"]] += stars
        self.mean = sum(y)/len(y)
        for city in star_sum:
            self.avg_stars[city] = star_sum[city]/count[city]
        return self
    
    
    def predict(self, X):
        for x in X:
            city = x["city"]
            if city not in self.avg_stars:
                x["pred"] = self.mean
            else:
                x["pred"] = self.avg_stars[city]
        return  [x["pred"] for x in X]
    
    

Now we can create an instance of our estimator and train it.

In [20]:
city_est = CityEstimator()
city_est.fit(data, star_ratings)

CityEstimator()

There is a problem, however. If we're asked to estimate the rating of a venue in a city that's not in our training set, the model will not perform well, so we came up with a function that we if the city is not in our list of city will return the average.

In [21]:
city_est.predict([{'city': 'Timbuktu'}])

[3.6729137013021247]

## lat_long_model

You can imagine that a city-based model might not be sufficiently fine-grained. For example, we know that some neighborhoods are trendier than others. We Use the latitude and longitude of a venue as features that helps us to understand neighborhood dynamics.

Instead of writing a custom estimator, we'll use one of the built-in estimators in Scikit Learn.  Since these estimators won't know what to do with a list of dictionaries, we'll build a `ColumnSelectTransformer` that will return an array containing selected keys of our feature matrix.  

In [22]:
class ColumnSelectTransformer(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self, col_names):
        self.col_names = col_names  
        self.data = {}
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):

        return [[row[name] for name in self.col_names] for row in X]

Let's test it on a single row, just as a check point:

In [23]:
cst = ColumnSelectTransformer(['latitude', 'longitude'])
assert (cst.fit_transform(data[:1])
        == [[data[0]['latitude'], data[0]['longitude']]])

Now, we use `sklearn.neighbors.KNeighborsRegressor`.  As a check point check, we'll test it with the first 5 rows.  To truly judge the performance, we'd need to make a test/train split.

In [24]:
from sklearn.neighbors import KNeighborsRegressor

data_transform = cst.fit_transform(data)
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(data_transform, star_ratings)
test_data = data[:5]
test_data_transform = cst.transform(test_data)
knn.predict(test_data_transform)

array([4. , 4.2, 4. , 3.8, 4.2])

Instead of doing this by hand, let's make a pipeline.

In [25]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
data_transform = cst.fit_transform(data)
pipe = Pipeline([
        # ColumnSelectTransformer
    ("Cst",ColumnSelectTransformer(['latitude', 'longitude'])),
        # KNeighborsRegressor
    ("KNN",KNeighborsRegressor(n_neighbors=46) )
    
    ])


We also will use Grid search to perform both crossvalidation and find the right values for the parameters of our model

In [26]:
param_grid = {"KNN__n_neighbors": [i*5+1 for i in range(10)]}
GridSCV = GridSearchCV(pipe, param_grid, cv=10)
GridSCV.fit(data, star_ratings)

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('Cst', ColumnSelectTransformer(col_names=['latitude', 'longitude'])), ('KNN', KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=46, p=2,
          weights='uniform'))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'KNN__n_neighbors': [1, 6, 11, 16, 21, 26, 31, 36, 41, 46]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [27]:
pipe.fit(data, star_ratings)
pipe.predict(test_data)

array([3.94565217, 3.77173913, 3.73913043, 3.61956522, 3.70652174])

## category_model

While location is important, we could also try seeing how predictive the
venue's category is. We will Build an estimator that considers only the categories.

The categories come as a list of strings, but the built-in estimators all need numeric input.  The standard way to deal with categorical features is **one-hot encoding**.

Scikit Learn provides [`DictVectorizer`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html#sklearn.feature_extraction.DictVectorizer), which takes in a list of dictionaries.  It creates a column in the output matrix for each key in the dictionary and fills it with the value associated with it.  Missing keys are filled with zeros.  Therefore, we need only build a transformer that takes a list of strings to a dictionary with keys given by those strings and values of one.

In [28]:
class DictEncoder(base.BaseEstimator, base.TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X): 
        return [{key:1 for key in cap[0]} for cap in X]
        

Check point

In [29]:
assert (DictEncoder().fit_transform([[['a']], [['b', 'c']]])
        == [{'a': 1}, {'b': 1, 'c': 1}])

Setting up a pipeline with `ColumnSelectTransformer`, your `DictEncoder`, the `DictVectorizer`, and a regularized linear model, like `Ridge`, as the estimator.  This model will have a large number of features, one for each category, so there is a significant danger of overfitting. We Use cross validation to choose the best regularization parameter.

In [30]:
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction import DictVectorizer


In [31]:
pipeCat = Pipeline([
        # ColumnSelectTransformer
    ("Cst",ColumnSelectTransformer(['categories'])),
    ("Dictencoder", DictEncoder()),
    ("DictVect", DictVectorizer()),
    ("TF-IDF", TfidfTransformer()),
    ("R", Ridge(alpha=1.0))
    ])

In [32]:
pipeCat.fit(data, star_ratings)

Pipeline(memory=None,
     steps=[('Cst', ColumnSelectTransformer(col_names=['categories'])), ('Dictencoder', DictEncoder()), ('DictVect', DictVectorizer(dtype=<class 'numpy.float64'>, separator='=', sort=True,
        sparse=True)), ('TF-IDF', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)), ('R', Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001))])

In [33]:
param_grid = {"R__alpha": [i*100 for i in range(10)]}
GridSCV = GridSearchCV(pipeCat, param_grid, cv=10)
GridSCV.fit(data, star_ratings)

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('Cst', ColumnSelectTransformer(col_names=['categories'])), ('Dictencoder', DictEncoder()), ('DictVect', DictVectorizer(dtype=<class 'numpy.float64'>, separator='=', sort=True,
        sparse=True)), ('TF-IDF', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)), ('R', Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'R__alpha': [0, 100, 200, 300, 400, 500, 600, 700, 800, 900]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

One step that can be taken toward improvement = Some categories (e.g. Restaurants) are not very specific.  Others (Japanese sushi) are much more so.  One way to deal with this is with an measure call term-frequency-inverse-document-frequency (TF-IDF). We can add in a `sklearn.feature_extraction.text.TfidfTransformer` between the `DictVectorizer` and the linear model, and see if that improves performance.

## attribute_model

There is even more information in the attributes for each venue.  Let's build an estimator based on these.

Venues attributes may be nested:
```
{
  'Attire': 'casual',
  'Accepts Credit Cards': True,
  'Ambiance': {'casual': False, 'classy': False}
}
```
We wish to encode them with one-hot encoding.  The `DictVectorizer` can do this, but only once we've flattened the dictionary to a single level, like so:
```
{
  'Attire_casual' : 1,
  'Accepts Credit Cards': 1,
  'Ambiance_casual': 0,
  'Ambiance_classy': 0
}
```

We Build a custom transformer that flattens the attributes dictionary and place it in a pipeline with a `DictVectorizer` and a regressor.

It might be difficult to find a single regressor that does well enough.  A common solution is to use a linear model to fit the linear part of some data, and use a non-linear model to fit the residual that the linear model can't fit.

In [36]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [34]:
class attrDictEncoder(base.BaseEstimator, base.TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    
    def FlattenDict(self, D):
        dic = {}
        for ins in D: 
            if type(D[ins]) is str:
                key = str(ins)+"_"+str(D[ins])
                value = 1
                dic.update({key: value})
            elif type(D[ins]) is bool:
                key = ins
                if D[ins] : 
                    dic.update({key: 1})
                else:
                    dic.update({key: 0})
            elif type(D[ins]) is dict:
                newdic = self.FlattenDict(D[ins])
                for d in newdic:
                    key = str(ins)+"_"+str(d)
                    value = newdic[d]
                    dic.update({key:value})
        return dic
    
    
    def transform(self, X): 
        flat= []
        for lis in X :
            flat.append(self.FlattenDict(lis[0]))
        return flat      

In [40]:
class ResidualEst(base.BaseEstimator, base.TransformerMixin):
    def __init__(self, baseEst, resEst):
        self.base = baseEst
        self.res = resEst
    
    def fit(self, X, y):
        self.base.fit(X, y)
        self.res.fit(X, y - self.base.predict(X))
    def predict(self, X):
        return self.base.predict(X)+ self.res.predict(X)
    
residual = ResidualEst(GridSearchCV(Ridge(), {"alpha":[i*5+1 for i in range(10)]}), GridSearchCV(RandomForestRegressor(max_depth = 100), {"min_samples_leaf":[2**i for i in range(0,5)]}))






In [41]:
##Checkpoint
x = [[{
  'Attire': 'casual',
  'Accepts Credit Cards': True,
  'Ambiance': {'casual': False, 'classy': False}
}],
[{
  'Attire': 'catastrophic',
  'Accepts Credit Cards': False,
  'Ambiance': {'casual': True, 'classy': False}
}]
]



In [42]:
cst = ColumnSelectTransformer(['attributes'])
Xtest1 = cst.fit_transform(data)

In [43]:
Xtest1

[[{'By Appointment Only': True}],
 [{'Take-out': True,
   'Good For': {'dessert': False,
    'latenight': False,
    'lunch': True,
    'dinner': False,
    'breakfast': False,
    'brunch': False},
   'Caters': False,
   'Noise Level': 'average',
   'Takes Reservations': False,
   'Delivery': False,
   'Ambience': {'romantic': False,
    'intimate': False,
    'touristy': False,
    'hipster': False,
    'divey': False,
    'classy': False,
    'trendy': False,
    'upscale': False,
    'casual': False},
   'Parking': {'garage': False,
    'street': False,
    'validated': False,
    'lot': True,
    'valet': False},
   'Has TV': True,
   'Outdoor Seating': False,
   'Attire': 'casual',
   'Alcohol': 'none',
   'Waiter Service': True,
   'Accepts Credit Cards': True,
   'Good for Kids': True,
   'Good For Groups': True,
   'Price Range': 1}],
 [{'Take-out': True,
   'Good For': {'dessert': False,
    'latenight': False,
    'lunch': False,
    'dinner': False,
    'breakfast': False,


In [44]:
y = [1,2,3,4,5,6]
del(y[0])
y
solver : {‘auto’, ‘svd’, ‘cholesky’, ‘lsqr’, ‘sparse_cg’, ‘sag’, ‘saga’}

SyntaxError: invalid character in identifier (<ipython-input-44-2132f24b08fe>, line 4)

In [45]:
pipeAttr = Pipeline([
        # ColumnSelectTransformer
    ("Cst",ColumnSelectTransformer(['attributes'])),
    ("Dictencoder", attrDictEncoder()),
    ("DictVect", DictVectorizer(sparse="False")),
    ("RS", residual)
    ])

In [46]:
pipeAttr.fit(data, star_ratings)

/Users/blackhawk/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/blackhawk/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/blackhawk/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/blackhawk/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 

Pipeline(memory=None,
     steps=[('Cst', ColumnSelectTransformer(col_names=['attributes'])), ('Dictencoder', attrDictEncoder()), ('DictVect', DictVectorizer(dtype=<class 'numpy.float64'>, separator='=', sort=True,
        sparse='False')), ('RS', ResidualEst(baseEst=None, resEst=None))])

In [91]:
GridSCV.best_params_

{'RF__n_estimators': 100}

## full_model

So far we have only built models based on individual features.  Now we will build an ensemble regressor that averages together the estimates of the four previous regressors.

In order to use the existing models as input to an estimator, we will have to turn them into transformers.  (A pipeline can contain at most a single estimator.) We will build a custom `ModelTransformer` class that takes an estimator as an argument.  When `fit()` is called, the estimator will be fit.  When `transform()` is called, the estimator's `predict()` method will be called, and its results returned.

Note that the output of the `transform()` method should be a 2-D array with a single column, in order for it to work well with the Scikit Learn pipeline. We ca use NumPy arrays `.reshape(-1, 1)` to create a column vector.

In [47]:
from numpy import array

In [48]:
class EstimatorTransformer(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self, estimator):
        self.estimator = estimator
    
    def fit(self, X, y):
        self.estimator.fit(X,y)
        return self
    
    def transform(self, X):
        return array(self.estimator.predict(X)).reshape(-1,1)
        

In [51]:
#Checkpoints
city_estim = CityEstimator()
city_trans = EstimatorTransformer(city_estim)
city_trans.fit(data, star_ratings)
assert ([r[0] for r in city_trans.transform(data[:5])]
        == city_estim.predict(data[:5]))

We create an instance of `ModelTransformer` for each of the previous four models and combine these together in a single feature matrix with a
[`FeatureUnion`](http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.FeatureUnion.html#sklearn.pipeline.FeatureUnion).

In [52]:
cityEst = EstimatorTransformer(CityEstimator())
Lt_lng = EstimatorTransformer(pipe)
Ctg = EstimatorTransformer(pipeCat)
Att = EstimatorTransformer(pipeAttr)

In [53]:
cty = EstimatorTransformer(CityEstimator())
cty.fit(data, star_ratings)


EstimatorTransformer(estimator=CityEstimator())

In [54]:
from sklearn.pipeline import FeatureUnion

union = FeatureUnion([
    ("CityEst", EstimatorTransformer(CityEstimator())),
    ("Lt_lng", EstimatorTransformer(pipe)),
    ("Ctg", EstimatorTransformer(pipeCat)),
    ("Att", EstimatorTransformer(pipeAttr))
    ])

In [55]:
#Checkpoint
union.fit(data, star_ratings)
trans_data = union.transform(data[:10])
assert trans_data.shape == (10, 4)

/Users/blackhawk/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/blackhawk/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/blackhawk/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/blackhawk/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 

In [243]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

Last_stand = Pipeline ([
    ("Union", union),
    ("LR", LinearRegression())
])

full_est = Last_stand.fit(data, star_ratings)

*Copyright &copy; 2019 The Data Incubator.  All rights reserved.*